In [1]:
import bs4, requests as req
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests

In [2]:
def land_on_books_main_page_js(driver):
    js_script = """var menu_tags = Array.from(document.getElementsByTagName('span'));
    menu_tags = menu_tags.filter(a=>a.innerText==='Sports, Books & More');
    menu_tags.forEach(function(tag){
    let tag_name = tag.textContent;
    if(tag_name==='Sports, Books & More'){
        tag.addEventListener('mouseover', function() {
        let books_tags = Array.from(document.getElementsByTagName('a'));
        books_tags.forEach(function(b_tag){
				if(b_tag.textContent==='Books'){
					b_tag.click();
				}
			});
		});
var event = new MouseEvent('mouseover', {
  'view': window,
  'bubbles': true,
  'cancelable': true
	});
tag.dispatchEvent(event);
	}
});
""";
    driver.execute_script(js_script);
    
def find_element_by_class_wait(driver,class_name):
    element = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.CLASS_NAME,class_name)));
    return element;

def find_elements_by_class_wait(driver,class_name):
    elements = WebDriverWait(driver,30).until(EC.presence_of_all_elements_located((By.CLASS_NAME,class_name)));
    return elements;

# method to click on 'show more' to expand the books genes list
def expand_genre_list(driver): 
    driver.execute_script("""showBtn = Array.from(document.getElementsByClassName('_3aoPnm')[0].getElementsByTagName('span')).filter(function(a){
	if(a.textContent){
		return a.textContent.startsWith('Show') && a.textContent.endsWith('more');
	}})[0];
    if(showBtn){
        showBtn.click();
    }
""")
    
def wait_till_page_loads_completely(driver):
    page_state = driver.execute_script('return document.readyState;');
    print('page_state',page_state)
    while page_state != 'complete':
        print('in wait method')
        time.sleep(1)

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path="chromedriver.exe",options=options)
driver.get('https://www.flipkart.com/')
driver.find_element_by_xpath("//button[@class='_2AkmmA _29YdH8']").click();

In [4]:
land_on_books_main_page_js(driver)
time.sleep(5)
expand_genre_list(driver)

In [5]:
#find the link for given genres
book_genre_a_tags = driver.find_elements_by_class_name('_2SvCnW')
req_genres = {'History and Archaeology Books':2000,
              'Literature Books':2000,'Healthy Living and Wellness Books':1000,
              'Biographies, Memoirs and General Non-Ficton Books':1000,
              'Self-Help Books':2000,'Philosophy and Religion Books':1000,
              'Arts, Language and Linguistic Books':1000,'Fiction Books':2000}

genre_and_links_and_count = list()
for tag in book_genre_a_tags:
    tag_name = tag.get_attribute('text')
    if(tag_name in list(req_genres.keys())):
        link = tag.get_attribute('href')
        gen_tuple = (tag_name,link,req_genres[tag_name])
        genre_and_links_and_count.append(gen_tuple)
genre_and_links_and_count

[('History and Archaeology Books',
  'https://www.flipkart.com/books/history-and-archaeology-books/pr?sid=bks,9xa&otracker=categorytree',
  2000),
 ('Literature Books',
  'https://www.flipkart.com/books/literature-books/pr?sid=bks,w4n&otracker=categorytree',
  2000),
 ('Healthy Living and Wellness Books',
  'https://www.flipkart.com/books/healthy-living-and-wellness-books/pr?sid=bks,4z4&otracker=categorytree',
  1000),
 ('Biographies, Memoirs and General Non-Ficton Books',
  'https://www.flipkart.com/books/biographies-memoirs-and-general-nonficton-books/pr?sid=bks,wmz&otracker=categorytree',
  1000),
 ('Self-Help Books',
  'https://www.flipkart.com/books/selfhelp-books/pr?sid=bks,5uw&otracker=categorytree',
  2000),
 ('Philosophy and Religion Books',
  'https://www.flipkart.com/books/philosophy-and-religion-books/pr?sid=bks,sqa&otracker=categorytree',
  1000),
 ('Arts, Language and Linguistic Books',
  'https://www.flipkart.com/books/arts-language-and-linguistic-books/pr?sid=bks,mzb&ot

In [6]:
def get_prod_links_in_page(link):
    resp =requests.get(link)
    soup = BeautifulSoup(resp.content, 'html.parser')
    all_products_a_tags = soup.find_all('a',{'class':'_2cLu-l'})
    prod_links = ['https://www.flipkart.com'+tag['href'] for tag in all_products_a_tags]
    return prod_links;

In [7]:
prod_links = dict()
for data_tuple in genre_and_links_and_count:
    genre_link = data_tuple[1]
    driver.get(genre_link);
    wait_till_page_loads_completely(driver)
    expand_genre_list(driver);
    sub_genre_a_tags = find_elements_by_class_wait(driver,'_2SvCnW')
    del sub_genre_a_tags[-1]
    sub_genre_links = [tag.get_attribute('href') for tag in sub_genre_a_tags]
    
    print(data_tuple[0],len(sub_genre_links))
    
    prod_links_by_genre = list()
    for s_g_link in sub_genre_links:
        no_iterations = round((data_tuple[2]/len(sub_genre_links))/40)
        no_iterations += 2
        no_iterations = 26 if no_iterations>26 else no_iterations
        print('no_iterations-%s, data_tuple[2]-%s' %(no_iterations,data_tuple[2]))
        for iteration in range(1,no_iterations):
            if iteration == no_iterations-1:
                print('iteration',iteration)
            link = s_g_link+'&page='+str(iteration)
            links = get_prod_links_in_page(link)
            prod_links_by_genre.extend(links)
    prod_links[data_tuple[0]]=prod_links_by_genre
#prod_links

page_state complete
History and Archaeology Books 2
no_iterations-26, data_tuple[2]-2000
iteration 25
no_iterations-26, data_tuple[2]-2000
iteration 25
page_state complete
Literature Books 3
no_iterations-19, data_tuple[2]-2000
iteration 18
no_iterations-19, data_tuple[2]-2000
iteration 18
no_iterations-19, data_tuple[2]-2000
iteration 18
page_state complete
Healthy Living and Wellness Books 2
no_iterations-14, data_tuple[2]-1000
iteration 13
no_iterations-14, data_tuple[2]-1000
iteration 13
page_state complete
Biographies, Memoirs and General Non-Ficton Books 2
no_iterations-14, data_tuple[2]-1000
iteration 13
no_iterations-14, data_tuple[2]-1000
iteration 13
page_state complete
Self-Help Books 3
no_iterations-19, data_tuple[2]-2000
iteration 18
no_iterations-19, data_tuple[2]-2000
iteration 18
no_iterations-19, data_tuple[2]-2000
iteration 18
page_state complete
Philosophy and Religion Books 2
no_iterations-14, data_tuple[2]-1000
iteration 13
no_iterations-14, data_tuple[2]-1000
iter

In [8]:
for k,v in prod_links.items():
    print(k,len(v))

History and Archaeology Books 2000
Literature Books 2160
Healthy Living and Wellness Books 1040
Biographies, Memoirs and General Non-Ficton Books 1040
Self-Help Books 2160
Philosophy and Religion Books 1040
Arts, Language and Linguistic Books 1000
Fiction Books 2640


In [9]:
driver.quit()

In [10]:
genre_size_dict = dict()
for i in genre_and_links_and_count:
    genre_size_dict[i[0]]=i[2]
    
for k,v in prod_links.items():
    with open('prod_urls/'+k+'.txt','w') as file:
        temp_v = v[:genre_size_dict[k]]
        print('temp_v',len(temp_v))
        for line in temp_v:
            file.write(line+'\n')

temp_v 2000
temp_v 2000
temp_v 1000
temp_v 1000
temp_v 2000
temp_v 1000
temp_v 1000
temp_v 2000
